In [ ]:
# Web dev: 
# Model relationships between a client, server, and database.

from rdflib import Graph, URIRef, Literal, RDF, RDFS

g = Graph()

client = URIRef("http://example.org/Client")
server = URIRef("http://example.org/Server")
database = URIRef("http://example.org/Database")

g.add((client, RDFS.label, Literal("Client")))
g.add((server, RDFS.label, Literal("Server")))
g.add((database, RDFS.label, Literal("Database")))
g.add((client, URIRef("http://example.org/interactsWith"), server))
g.add((server, URIRef("http://example.org/interactsWith"), database))

print(g.serialize(format="turtle").decode("utf-8"))

In [ ]:
# Healthcare:
# Model relationships between diseases, symptoms, and treatments.

disease = URIRef("http://example.org/Disease")
symptom = URIRef("http://example.org/Symptom")
treatment = URIRef("http://example.org/Treatment")

g.add((disease, RDFS.label, Literal("Flu")))
g.add((symptom, RDFS.label, Literal("Fever")))
g.add((treatment, RDFS.label, Literal("Antiviral Medication")))
g.add((disease, URIRef("http://example.org/hasSymptom"), symptom))
g.add((disease, URIRef("http://example.org/treatedBy"), treatment))

print(g.serialize(format="turtle").decode("utf-8"))


In [ ]:
# Academic Research
# Model relationships between authors, papers, and citations

author = URIRef("http://example.org/Author")
paper = URIRef("http://example.org/Paper")
citation = URIRef("http://example.org/Citation")

g.add((author, RDFS.label, Literal("Alice")))
g.add((paper, RDFS.label, Literal("AI Research Paper")))
g.add((citation, RDFS.label, Literal("Cited by Bob")))
g.add((author, URIRef("http://example.org/wrote"), paper))
g.add((paper, URIRef("http://example.org/citedBy"), citation))

print(g.serialize(format="turtle").decode("utf-8"))

In [ ]:
# Smart Cities
# Model relationships between buildings, sensors, and data

building = URIRef("http://example.org/Building")
sensor = URIRef("http://example.org/Sensor")
data = URIRef("http://example.org/Data")

g.add((building, RDFS.label, Literal("Office Building")))
g.add((sensor, RDFS.label, Literal("Temperature Sensor")))
g.add((data, RDFS.label, Literal("Temperature Data")))
g.add((building, URIRef("http://example.org/hasSensor"), sensor))
g.add((sensor, URIRef("http://example.org/generates"), data))

print(g.serialize(format="turtle").decode("utf-8"))

In [ ]:
# Social Networks
# Model relationships between users, posts, and comments

user = URIRef("http://example.org/User")
post = URIRef("http://example.org/Post")
comment = URIRef("http://example.org/Comment")

g.add((user, RDFS.label, Literal("John Doe")))
g.add((post, RDFS.label, Literal("Python Tutorial Post")))
g.add((comment, RDFS.label, Literal("Great article!")))
g.add((user, URIRef("http://example.org/authored"), post))
g.add((post, URIRef("http://example.org/hasComment"), comment))

print(g.serialize(format="turtle").decode("utf-8"))

In [1]:
# Querying a Simple Ontology
# SPARQL is used to query RDF data stored in ontologies. Python libraries like rdflib and SPARQLWrapper are ideal for this.

from rdflib import Graph, Namespace
from rdflib.plugins.sparql import prepareQuery

# Define a small RDF graph
g = Graph()
EX = Namespace("http://example.org/")
g.bind("ex", EX)

# Add triples to the ontology
g.add((EX.Client, EX.interactsWith, EX.Server))
g.add((EX.Server, EX.interactsWith, EX.Database))

# SPARQL query to find all interactions
query = prepareQuery("""
    SELECT ?subject ?object
    WHERE {
        ?subject <http://example.org/interactsWith> ?object.
    }
""")

# Execute query
for row in g.query(query):
    print(f"{row.subject} interacts with {row.object}")

In [ ]:
# Querying with SPARQL

from rdflib import Graph, Namespace
from rdflib.plugins.sparql import prepareQuery

# Define RDF graph
g = Graph()
EX = Namespace("http://example.org/")
g.bind("ex", EX)

# Add triples to ontology
g.add((EX.Person, EX.hasName, EX.Alice))
g.add((EX.Alice, EX.hasFriend, EX.Bob))

# SPARQL query: Find all friends of Alice
query = prepareQuery("""
    SELECT ?friend
    WHERE {
        <http://example.org/Alice> <http://example.org/hasFriend> ?friend.
    }
""")

# Execute query
for row in g.query(query):
    print(f"Alice's friend: {row.friend}")



In [ ]:
# Validating Ontologies Using SHACL
# SHACL (Shapes Constraint Language) ensures RDF data conforms to a specific schema or set of rules. Python’s pyshacl library allows SHACL validation.

from rdflib import Graph
from pyshacl import validate

# Define RDF graph
data_graph = Graph()
data_graph.parse(data="""
    @prefix ex: <http://example.org/> .
    ex:Person ex:hasName "Alice" .
""", format="ttl")

# Define SHACL shapes graph
shapes_graph = Graph()
shapes_graph.parse(data="""
    @prefix sh: <http://www.w3.org/ns/shacl#> .
    @prefix ex: <http://example.org/> .

    ex:PersonShape
        a sh:NodeShape ;
        sh:targetClass ex:Person ;
        sh:property [
            sh:path ex:hasName ;
            sh:datatype xsd:string ;
        ] .
""", format="ttl")

# Validate
conforms, report_graph, report_text = validate(data_graph, shacl_graph=shapes_graph)
print(f"Conforms: {conforms}")
print(report_text)

In [ ]:
# Creating and Querying a SKOS Ontology
# SKOS (Simple Knowledge Organization System) is used to create controlled vocabularies or hierarchical taxonomies. Python can manipulate SKOS structures to manage concepts and relationships.

from rdflib import Graph, Namespace

# Define namespaces
SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
EX = Namespace("http://example.org/")

# Create RDF graph
g = Graph()
g.bind("skos", SKOS)
g.bind("ex", EX)

# Add SKOS concepts
g.add((EX.Fruit, SKOS.prefLabel, Literal("Fruit")))
g.add((EX.Apple, SKOS.broader, EX.Fruit))
g.add((EX.Banana, SKOS.broader, EX.Fruit))
g.add((EX.Fruit, SKOS.narrower, EX.Apple))
g.add((EX.Fruit, SKOS.narrower, EX.Banana))

# Query SKOS relationships
query = """
    SELECT ?concept ?broader
    WHERE {
        ?concept <http://www.w3.org/2004/02/skos/core#broader> ?broader.
    }
"""
for row in g.query(query):
    print(f"Concept: {row.concept}, Broader term: {row.broader}")



In [ ]:
# Validating SKOS Ontologies with SHACL
# You can combine SKOS and SHACL to validate that concepts in your ontology follow SKOS rules.

from pyshacl import validate

# SKOS data
data_graph = Graph()
data_graph.parse(data="""
    @prefix skos: <http://www.w3.org/2004/02/skos/core#> .
    @prefix ex: <http://example.org/> .

    ex:Fruit a skos:Concept ;
        skos:prefLabel "Fruit"@en ;
        skos:narrower ex:Apple, ex:Banana .

    ex:Apple a skos:Concept ;
        skos:prefLabel "Apple"@en ;
        skos:broader ex:Fruit .
""", format="ttl")

# SHACL validation rules
shapes_graph = Graph()
shapes_graph.parse(data="""
    @prefix sh: <http://www.w3.org/ns/shacl#> .
    @prefix skos: <http://www.w3.org/2004/02/skos/core#> .

    ex:SKOSConceptShape
        a sh:NodeShape ;
        sh:targetClass skos:Concept ;
        sh:property [
            sh:path skos:prefLabel ;
            sh:datatype xsd:string ;
        ] .
""", format="ttl")

# Validate
conforms, report_graph, report_text = validate(data_graph, shacl_graph=shapes_graph)
print(f"Conforms: {conforms}")
print(report_text)

In [ ]:
# Accessing External Ontologies
# Python can interact with external SPARQL endpoints to query large ontologies, like DBpedia or Wikidata, using SPARQLWrapper.

from SPARQLWrapper import SPARQLWrapper, JSON

# Define SPARQL endpoint
sparql = SPARQLWrapper("https://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?fruit ?label
    WHERE {
        ?fruit a dbo:Fruit .
        ?fruit rdfs:label ?label .
        FILTER (lang(?label) = 'en')
    } LIMIT 5
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Print results
for result in results["results"]["bindings"]:
    print(f"Fruit: {result['fruit']['value']}, Label: {result['label']['value']}")